In [ ]:
import geopandas as gpd
from shapely import geometry
import pyproj
import matplotlib.pyplot as plt

# GeoPandas数据IO  

一般来说，矢量数据都通过shapefile等格式进行传输和存储，GeoPandas也提供了对这些文件进行读取和输出的能力

## 读取矢量数据

GeoPandas提供了`geopandas.read_file()`函数来读取各类矢量数据，并转换为GeoDataframe数据结构  

实际上，`geopandas.read_file()`函数由[Fiona](https://pypi.org/project/Fiona/)的矢量API实现

下面将举例一些常用文件格式的矢量数据的读取

### 读取shapefile

In [ ]:
# 直接在geopandas.read_file()中传入路径即可：
shpdata = gpd.read_file('data/hangzhou_shp/hangzhou.shp')

In [ ]:
shpdata.head()

In [ ]:
# 也可以直接读取zip包中的shapefile

# shapefile文件在压缩包根目录
zipdata1 = gpd.read_file('zip://data/县.zip')
zipdata1.head()

In [ ]:
# shapefile文件在压缩包子目录下时，可以使用英文感叹号!做分割，在感叹后输入子路径
zipdata2 = gpd.read_file('zip://data/hangzhou.zip!WGS84/hangzhou.shp')
zipdata2.head()

### 读取GeoJson

In [ ]:
# 直接在geopandas.read_file()中传入路径即可：
gjsondata = gpd.read_file('data/hangzhou.geojson')

In [ ]:
gjsondata.head()

### 读取gdb和gpkg

ESRI (ArcGIS) 的地理数据库文件`gdb`与OGC (QGIS) 的地理包文件`gpkg`均基于图层结构对矢量数据进行组织，因此在读取时需要增加`layer`参数标明具体图层

下面以gpkg文件为例：

In [ ]:
# 全国区划.gpkg 包括了省、市两个图层，因此在读取时需要指定具体图层：
gpkgdata_province = gpd.read_file('data/全国区划.gpkg',layer='省')
gpkgdata_city = gpd.read_file('data/全国区划.gpkg',layer='市')

In [ ]:
gpkgdata_province.head()

In [ ]:
gpkgdata_city.head()

### 读取带点坐标的csv

带坐标的csv文件严格说来与之前的文件都不同，使用`read_file()`读取后得到的是一个不含几何列的geodataframe，还需要对经纬度字段进行处理，生成真正可用的geodataframe

下面是一种可行的方式：

In [ ]:
csvdf= gpd.read_file(r'./data/sample_point.csv',encoding='utf-8')    # 读取数据
csvgdf = gpd.GeoDataFrame(csvdf, geometry=gpd.points_from_xy(csvdf.lng, csvdf.lat))    # 构造几何列
csvgdf.crs = pyproj.CRS.from_user_input('EPSG:4326')    # 指定投影
csvgdf.head()

In [ ]:
%matplotlib widget
ax = csvgdf.plot()

## 过滤

read_file()函数在读取数据时，允许增加一定的过滤条件，只提取部分数据进入dataframe。过滤的方式主要有：

- bbox
- mask
- rows

### bbox

`bbox` 过滤采用一个矩形范围对数据进行过滤，与bbox在空间上相交的数据将被读取进入dataframe

In [ ]:
bboxdata = gpd.read_file('zip://data/县.zip',
                         bbox=(118.333,29.195,120.927,30.556))
bboxdata.tail()

In [ ]:
%matplotlib widget
ax = bboxdata.plot()
# 绘制bbox框示意
ax = gpd.GeoSeries([geometry.box(minx=118.333, 
                                 miny=29.195, 
                                 maxx=120.927, 
                                 maxy=30.556).boundary]).plot(ax=ax, color='red')

### mask

`mask`与`bbox`类似，不过可以传入更为复杂的多边形来进行数据过滤

In [ ]:
maskdata = gpd.read_file('zip://data/县.zip',
                         mask=geometry.Polygon([(120.927,30.556),(119.671,30.642),(118.333,29.195)]))
maskdata.tail()

In [ ]:
%matplotlib widget
ax = maskdata.plot()
# 绘制bbox框示意
ax = gpd.GeoSeries([geometry.LinearRing([(120.927,30.556),(119.671,30.642),(118.333,29.195)])]).plot(ax=ax, color='red')
ax = bboxdata.plot(ax=ax,alpha=0.5)

### rows

`rows`有些类似于我们之前一直在用的`head()`函数，rows可以过滤前若干行数据进行读取，可以用来对大型数据进行试验性质的读取

In [ ]:
rowsdata = gpd.read_file('zip://data/县.zip',
                         rows = 5)
rowsdata

## 输出矢量数据

`to_file()` 函数可以用来将GeoDataframe和GeoSeries中的矢量数据输出

除了gdb由于是ESRI未开源的私有格式而无法导出外，shapefile、geojson、geopackage都可以由GeoPandas输出

### shapefile & geojson

In [ ]:
import os

try:
    os.mkdir('output')
except FileExistsError:
    pass
# 将刚刚转换的csv输出到shapefile
csvdf.to_file('data/output/csv2shp.shp', driver='ESRI Shapefile', encoding='utf-8')
csvdf.to_file('data/output/csv2json.geojson', driver='GeoJSON', encoding='utf-8')

### geopackage

In [ ]:
# geopackage和读取时一样，需要多一个layer参数
csvdf.to_file('data/output/csv2gpkg.gpkg', driver='GPKG',layer='layer1', encoding='utf-8')

# 数据投影

我们都知道地理数据的分析处理离不开坐标系的支持，理所当然地，GeoPandas基于 [pyproj](https://pyproj4.github.io/pyproj/stable/) 包提供了对于坐标参考系（CRS）的一系列操作

## 读取投影

GeoDataFrame的crs属性记录了这份数据的坐标参考系

In [ ]:
shpdata = gpd.read_file('data/hangzhou_shp/hangzhou.shp')
shpdata.crs

## 定义投影

有些数据可能不包含坐标系，在读取数据后可以手动为其定义投影信息

In [ ]:
data = gpd.read_file('zip://data/hangzhou_raw.zip')    # 该压缩包内缺少.prj文件
print(data.crs)

In [ ]:
# 使用pyproj包可以为geodataframe定义投影
import pyproj

data.crs = pyproj.CRS.from_user_input('EPSG:4326')
print(data.crs)

## 重投影

`to_crs()`提供了将几何数据从当前坐标参考系投影到指定坐标参考系的能力

In [ ]:
shpdata = gpd.read_file('data/hangzhou_shp/hangzhou.shp')
shpdata_proj =  shpdata.to_crs(crs='EPSG:32651')
%matplotlib widget
ax = shpdata_proj.plot()

# 基础可视化

GeoPandas基于[matplotlib](https://matplotlib.org/)包实现了基本的可视化功能：GeoSeries和GeoDataFrame对象通过调用plot()方法即可绘制基本图样。

## GeoSeries 可视化

In [ ]:
china_province = gpd.read_file('data/全国区划.gpkg',layer='省')
china_nine = gpd.read_file('data/southsea/china_nine_dotted_line.shp')
china_capital = gpd.read_file('data/全国区划.gpkg',layer='省会')

In [ ]:
# 直接使用plot()就可以实现最简单的可视化
china_province.geometry.plot()

In [ ]:
# 通过设定 ax 参数，可以将多个数据叠加绘制
ax = china_province.geometry.plot()
china_nine.geometry.plot(ax=ax)

In [ ]:
# facecolor、edgecolor、alpha三个参数可以控制可视化的色彩
ax = china_province.geometry.plot(facecolor='grey', edgecolor='white')
china_nine.geometry.plot(ax=ax, edgecolor = 'grey', alpha = 0.6)

In [ ]:
# 上图的省界似乎有点太粗了，我们可以用linestyle和linewidth进行一些修改
ax = china_province.geometry.plot(facecolor='grey', edgecolor='white',
                                  linewidth=0.5, linestyle='-.')
china_nine.geometry.plot(ax=ax, edgecolor = 'grey', alpha = 0.6)

In [ ]:
# hatch参数可以对面要素的填充进行纹理化
ax = china_province.geometry.plot(facecolor='grey', edgecolor='white',
                                  linewidth=0.5, linestyle='-.', hatch='....')
china_nine.geometry.plot(ax=ax, edgecolor = 'grey', alpha = 0.6)

In [ ]:
# marker参数可以对点要素的符号进行设定，markersize对其大小进行设定
ax = china_province.geometry.plot(facecolor='grey', edgecolor='white',
                                  linewidth=0.5, linestyle='-.')
china_nine.geometry.plot(ax=ax, edgecolor = 'grey', alpha = 0.6)
china_capital.geometry.plot(ax=ax, edgecolor='black',
                            marker='^',markersize=6)

In [ ]:
# label参数可以设定该图层在图例中显示的名称
ax = china_province.geometry.plot(facecolor='grey', edgecolor='white',
                                  linewidth=0.5, linestyle='-.')
china_nine.geometry.plot(ax=ax, edgecolor = 'grey', alpha = 0.6,
                         label = '九段线')
china_capital.geometry.plot(ax=ax, edgecolor='black',
                            marker='^',markersize=6,
                            label = '省会')
#绘制图例
plt.rcParams["font.family"] = "SimHei"    # matplotlib支持的中文字体，不设置会乱码
ax.legend(title='图例',loc='lower left', ncol=1)

In [ ]:
# 使用matplotlib的add_axes()函数可以设置子图
# add_axes(x1,y1,x2,y2)中，x1,y1是子图左下角在主图的比例位置；x2,y2是子图宽高与主图宽高的比值

fig = plt.figure(figsize=(7,4))    # 配置画布

ax = fig.add_axes((0,0,1,1))       # 配置主坐标轴
# 绘制主画布的内容
ax = china_province.geometry.plot(ax = ax,facecolor='grey', edgecolor='white',
                                  linewidth=0.5, linestyle='-.')
china_capital.geometry.plot(ax=ax, edgecolor='black',
                            marker='^',markersize=6,
                            label = '省会')
ax = china_nine.geometry.plot(ax =ax, edgecolor = 'grey', alpha = 0.6,
                               label = '九段线')
plt.rcParams["font.family"] = "SimHei"
ax.legend(title='图例',loc='lower left', ncol=1)
# 设置范围限定
ax.set_xlim(70, 140)
ax.set_ylim(15, 55)

# 绘制子画布的内容
vax = fig.add_axes([0.8, 0.1, 0.2, 0.25])    # 配置子坐标轴
vax = china_nine.geometry.plot(ax =vax, edgecolor = 'grey', alpha = 0.6,
                               label = '九段线')
# 设置范围限定
vax.set_xlim(106.5, 123)
vax.set_ylim(2.8, 24.5)

## GeoDataFrame 可视化

GeoDataFrame由于带有了属性数据，因此可以做一些更复杂的可视化方式，比如分层设色

In [ ]:
china_province = gpd.read_file('data/全国区划.gpkg',layer='省')

In [ ]:
# 我们可以简单地将需要设色的字段设置为column，并用cmap指定设色的色带
# 色带：https://matplotlib.org/3.5.1/tutorials/colors/colormaps.html
fig = plt.figure(figsize=(7,6))
ax = fig.add_axes((0,0,1,1))
ax = china_province.plot(ax=ax,facecolor='grey', edgecolor='white',
                                  linewidth=0.5, linestyle='-.',
                                  column='人口',cmap='GnBu'
                                  )
ax = china_nine.geometry.plot(ax =ax, edgecolor = 'grey', alpha = 0.6,
                               label = '九段线')

In [ ]:
# 上一张图显然是有问题的，由于第七次人口普查没有港澳台的人口数据，可视化结果里缺失了他们的信息
# 我们可以使用missing_kwds参数对缺失数据进行处理
fig = plt.figure(figsize=(7,6))
ax = fig.add_axes((0,0,1,1))
ax = china_province.plot(ax=ax,facecolor='grey', edgecolor='white',
                         linewidth=0.5, linestyle='-.',
                         column='人口',cmap='GnBu',
                         missing_kwds={
                             'color':'grey',
                             'hatch':'////'
                         }
                         )
ax = china_nine.geometry.plot(ax =ax, edgecolor = 'grey', alpha = 0.6,
                               label = '九段线')

In [ ]:
# legend参数可以提供色带的图例
fig = plt.figure(figsize=(9,6))
ax = fig.add_axes((0,0,1,1))
ax = china_province.plot(ax=ax,facecolor='grey', edgecolor='white',
                         linewidth=0.5, linestyle='-.',
                         column='人口',cmap='GnBu',
                         missing_kwds={
                             'color':'grey',
                             'hatch':'////'
                         },
                         legend=True
                         )
ax = china_nine.geometry.plot(ax =ax, edgecolor = 'grey', alpha = 0.6,
                               label = '九段线')

In [ ]:
# scheme和k参数可以设定分类方式和类别数目
fig = plt.figure(figsize=(9,6))
ax = fig.add_axes((0,0,1,1))
ax = china_province.plot(ax=ax,facecolor='grey', edgecolor='white',
                         linewidth=0.5, linestyle='-.',
                         column='人口',cmap='GnBu',
                         missing_kwds={
                             'color':'grey',
                             'hatch':'////'
                         },
                         scheme='NaturalBreaks',k=5,
                         legend=True, 
                         legend_kwds={
                             'loc':'lower left'
                         }
                         )
ax = china_nine.geometry.plot(ax =ax, edgecolor = 'grey', alpha = 0.6,
                               label = '九段线')